In [ ]:
# MERGE ALL TEMP .H5 FILES INTO A SINGLE FINAL FILE


import csv, ast, os
from collections import defaultdict
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
import pandas as pd

# === Step 6: Patch to merge all .h5 files ===
def merge_all_hdf5_files(final_file="final_tick_data.h5"):
    temp_files = [f for f in os.listdir(".") if f.endswith(".h5") and f != final_file]
    with pd.HDFStore(final_file, mode='a') as final_store:
        for temp in temp_files:
            try:
                with pd.HDFStore(temp, mode='r') as temp_store:
                    keys = temp_store.keys()
                    if not keys:
                        print(f"⚠️ Skipping empty file: {temp}")
                        continue
                    for key in keys:
                        df = temp_store[key]
                        final_store.put(key, df, format='table', data_columns=True)
            except Exception as e:
                print(f"❌ Error reading {temp}: {e}")
            finally:
                try:
                    os.remove(temp)
                    print(f"🗑️ Deleted temp file: {temp}")
                except Exception as e:
                    print(f"⚠️ Could not delete {temp}: {e}")

merge_all_hdf5_files()

In [3]:
# DECOMPOSE FINAL FILE BY INSTRUMENT


import csv, ast, os
from collections import defaultdict
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
import pandas as pd

def decompose_by_instrument(final_file, output_dir="split_by_instrument"):
    os.makedirs(output_dir, exist_ok=True)
    with pd.HDFStore(final_file, mode='r') as store:
        keys = store.keys()
        instrument_groups = defaultdict(list)
        for key in keys:
            instrument = key.strip("/").split("/")[0]
            instrument_groups[instrument].append(key)

        for instrument, group_keys in instrument_groups.items():
            out_path = os.path.join(output_dir, f"{instrument}_tick_data.h5")
            with pd.HDFStore(out_path, mode='w') as out_store:
                for key in group_keys:
                    df = store[key]
                    out_store.put(key, df, format='table', data_columns=True)

decompose_by_instrument("final_tick_data.h5")
print("✅ All tasks complete. Data split by instrument.")

✅ All tasks complete. Data split by instrument.


In [5]:
# MERGE TO RAW FILES

import os
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

def merge_instrument_file(instrument, fetched_dir="split_by_instrument", raw_dir="./2015_tick_data"):
    fetched_path = os.path.join(fetched_dir, f"{instrument}_tick_data.h5")
    raw_path = os.path.join(raw_dir, f"{instrument}_tick_data.h5")

    if not os.path.exists(fetched_path):
        print(f"⚠️ Fetched file missing: {instrument}")
        return
    if not os.path.exists(raw_path):
        print(f"⚠️ Raw file missing: {instrument}")
        return

    with pd.HDFStore(raw_path, mode='a') as raw_store, pd.HDFStore(fetched_path, mode='r') as fetched_store:
        for key in fetched_store.keys():
            if key in raw_store:
                print(f"🔁 Skipping duplicate key: {key} in {instrument}")
                continue
            df = fetched_store[key]
            raw_store.put(key, df, format='table', data_columns=True)

    print(f"✅ Merged fetched → raw: {instrument}")

def main():
    fetched_dir = "split_by_instrument"     # Correct folder for fetched files
    raw_dir = "./2015_tick_data"            # Correct folder for raw files

    # Get list of instruments based on fetched files
    instruments = [
        filename.replace("_tick_data.h5", "")
        for filename in os.listdir(fetched_dir)
        if filename.endswith("_tick_data.h5")
    ]

    # Run merge in parallel
    with ThreadPoolExecutor(max_workers=32) as executor:
        for instrument in instruments:
            executor.submit(merge_instrument_file, instrument, fetched_dir, raw_dir)

if __name__ == "__main__":
    main()

🔁 Skipping duplicate key: /ausidxaud/y2023/m07/d30 in ausidxaud
🔁 Skipping duplicate key: /ausidxaud/y2023/m03/d05 in ausidxaud
🔁 Skipping duplicate key: /eurusd/y2023/m09/d03 in eurusd
🔁 Skipping duplicate key: /eurusd/y2023/m03/d05 in eurusd
✅ Merged fetched → raw: eurusd
🔁 Skipping duplicate key: /usa500idxusd/y2024/m01/d01 in usa500idxusd
🔁 Skipping duplicate key: /usdcnh/y2024/m05/d19 in usdcnh
🔁 Skipping duplicate key: /usdcnh/y2022/m06/d05 in usdcnh
🔁 Skipping duplicate key: /usdcnh/y2019/m05/d26 in usdcnh
✅ Merged fetched → raw: usdcnh
🔁 Skipping duplicate key: /usdchf/y2025/m07/d20 in usdchf
🔁 Skipping duplicate key: /usdmxn/y2023/m11/d19 in usdmxn
🔁 Skipping duplicate key: /usdsek/y2025/m04/d06 in usdsek
🔁 Skipping duplicate key: /usdsek/y2025/m02/d09 in usdsek
🔁 Skipping duplicate key: /usdsek/y2025/m02/d16 in usdsek
🔁 Skipping duplicate key: /gbpusd/y2025/m02/d23 in gbpusd
🔁 Skipping duplicate key: /gbpusd/y2024/m02/d18 in gbpusd
🔁 Skipping duplicate key: /gbpusd/y2023/m11/